In [4]:
from data_structures.graphs import *
from algos.graphs import *
from data_structures.heaps import *
from heapq import heapify,heappush,heappop
import sys
%load_ext autoreload

In [5]:
%autoreload
g1 = build_graph_savage_bfs()
g2 = build_graph_savage_dfs()

In [9]:
def DFS(g):
    visited = {v:False for v in g.graph.keys()}
    dfs_tree = {v:None for v in g.graph.keys()}
    top_sort = []
    
    def DFS_util(u,visited,dfs_tree,top_sort):
        visited[u] = True
        for v in g.graph[u]:
            if visited[v] is False:
                dfs_tree[v] = u
                DFS_util(v,visited,dfs_tree,top_sort)
        top_sort.append(u)
    
    for u in g.graph.keys():
        if visited[u] is False:
            dfs_tree[u] = u
            DFS_util(u,visited,dfs_tree,top_sort)
            
    return dfs_tree,top_sort

In [10]:
def BFS(g):
    visited = {v:False for v in g.graph.keys()}
    bfs_tree = {v:None for v in g.graph.keys()}
    q=[]
    
    def BFS_util(visited,bfs_tree,q):
        while len(q)>0:
            u = q.pop(0)
            for v in g.graph[u]:
                if visited[v] is False:
                    q.append(v)
                    visited[v] = True
                    bfs_tree[v] = u
        
    for u in g.graph.keys():
        if visited[u] is False:
            bfs_tree[u] = u
            q.append(u)
            visited[u] = True
            BFS_util(visited,bfs_tree,q)
    
    return bfs_tree

In [12]:
dfs_tree,top_sort = DFS(g2)
print(dfs_tree)
print(top_sort)

{1: 1, 3: 1, 5: 7, 6: 1, 9: 3, 2: 3, 4: 2, 7: 2, 8: 8}
[4, 5, 7, 2, 9, 3, 6, 1, 8]


In [13]:
bfs_tree=BFS(g1)
print(bfs_tree)

{'r': 'r', 's': 'r', 'v': 'r', 'w': 's', 't': 'w', 'u': 't', 'x': 'w', 'y': 'x'}


In [18]:
def invert(g):
    invg = Graph()
    for u in g.graph.keys():
        for v in g.graph[u]:
            invg.graph[v].append(u)
    return invg

In [19]:
g1i = invert(g1)
print(g1i.graph)

defaultdict(<class 'list'>, {'s': ['r', 'w'], 'v': ['r'], 'r': ['s', 'v'], 'w': ['s', 't', 'x'], 't': ['w', 'u', 'x'], 'x': ['w', 't', 'y'], 'u': ['t', 'y'], 'y': ['u', 'x']})


### Minimum Spanning Tree

In [20]:
def mst_graph():
    g = Graph()
    edges = [
        ('a','u'),
        ('a','v'),
        ('a','b'),
        ('b','v'),
        ('b','w'),
        ('b','c'),
        ('c','w'),
        ('c','x'),
        ('c','d'),
        ('d','x'),
        ('u','v'),
        ('v','w'),
        ('w','x'),
    ]
    wts=[1,1,2,6,3,5,6,4,5,3,1,2,4]
    g.create_from_edgelist(edges,wts)
    
    return g

In [21]:
g = mst_graph()

In [22]:
g.graph

defaultdict(list,
            {'a': ['u', 'v', 'b'],
             'u': ['a', 'v'],
             'v': ['a', 'b', 'u', 'w'],
             'b': ['a', 'v', 'w', 'c'],
             'w': ['b', 'c', 'v', 'x'],
             'c': ['b', 'w', 'x', 'd'],
             'x': ['c', 'd', 'w'],
             'd': ['c', 'x']})

In [28]:
def kruskal_mst(g):
    es = g.edgelist()
    nv = len(g.graph.keys())
    
    sorted_es = sorted(es,key=lambda x:g.weights[x])
    ds = {v:v for v in g.graph.keys()}
    
    def find_set(u):
        if u == ds[u]:
            return u
        else:
            ds[u] = find_set(ds[u])
            return ds[u]
    
    def union(us, vs):
        ds[vs] = us
    
    te = []
    we = []
    for e in sorted_es:
        u,v = e
        us = find_set(u)
        vs = find_set(v)
        if us is not vs:
            te.append(e)
            we.append(g.weights[e])
            union(us,vs)
        if len(te) == nv-1:
            break
    
    tree = Graph()
    tree.create_from_edgelist(te, we)
    return tree

In [29]:
tree = kruskal_mst(g)

In [30]:
tree.graph

defaultdict(list,
            {'a': ['u', 'v', 'b'],
             'u': ['a'],
             'v': ['a', 'w'],
             'b': ['a'],
             'w': ['v', 'x'],
             'd': ['x'],
             'x': ['d', 'w', 'c'],
             'c': ['x']})

In [9]:
def prims_mst(g):
    hp = MinHeap()
    par = {}
    inheap = {}
    for v in g.graph.keys():
        par[v] = v
        hp.insert([v,sys.maxsize])
        inheap[v] = True
        
    r = list(g.graph.keys())[3]
    hp.decrease_key(r, 0)
    #par[r] = r
    
    te, wts = [],[]
    while not hp.is_empty():
        u,w = hp.extract_min()
        inheap[u] = False
        te.append((par[u], u))
        wts.append(w)
        
        for v in g.graph[u]:
            if inheap[v] and g.weights[(u,v)] < hp.get_value(v):
                par[v] = u
                hp.decrease_key(v,g.weights[(u,v)])
                
    tree = Graph()
    tree.create_from_edgelist(te[1:], wts[1:])
    return tree
        

In [10]:
%autoreload
tree = prims_mst(g)

In [11]:
tree.graph

defaultdict(list,
            {'b': ['a'],
             'a': ['b', 'u', 'v'],
             'u': ['a'],
             'v': ['a', 'w'],
             'w': ['v', 'x'],
             'x': ['w', 'd', 'c'],
             'd': ['x'],
             'c': ['x']})

### Shortest Paths

In [36]:
def dijkstra_sssp(g,src):
    """
    Determine shortest paths from a given single source 'src', for a g that may be dir/undir 
    but always have positive weights.
    """
    d = {v: sys.maxsize for v in g.graph.keys()}
    par = {v:None for v in g.graph.keys()}
    S = set() #set of nodes whose shortest paths have been found
    
    hp = MinHeap()
    for v in g.graph.keys():
        hp.insert([v,d[v]])
    
    par[src] = src
    d[src] = 0
    hp.decrease_key(src,d[src])
    
    te, wts = [], []
    while not hp.is_empty():
        u, du = hp.extract_min()
        S.add(u)
        te.append((par[u], u))
        wts.append(du)
        
        for v in g.graph[u]:
            if v not in S:
                if d[u] + g.weights[(u,v)] < d[v]:
                    d[v] = d[u] + g.weights[(u,v)] 
                    par[v] = u
                    hp.decrease_key(v, d[v])
    
    def get_path1(u, p):
        if u == src:
            p.append(src)
            return p[::-1]
        else:
            p.append(u)
            return get_path(par[u],p)
    
    def get_path(u,src,par):
        if u == src:
            return [u]
        else:
            return get_path(par[u],src,par) + [u]
    
    paths = {}
    for v in list(S):
        paths[(src,v)] = get_path(v,src,par)
        
    return paths


In [37]:
paths = dijkstra_sssp(g, 'a')

In [38]:
paths

{('a', 'u'): ['a', 'u'],
 ('a', 'x'): ['a', 'v', 'w', 'x'],
 ('a', 'd'): ['a', 'v', 'w', 'x', 'd'],
 ('a', 'c'): ['a', 'b', 'c'],
 ('a', 'v'): ['a', 'v'],
 ('a', 'b'): ['a', 'b'],
 ('a', 'w'): ['a', 'v', 'w'],
 ('a', 'a'): ['a']}

In [27]:
def bellman_ford_sssp(g, src):
    """
    Bellman-Ford Single Source Shortest Path Algorithm, which works for graphs with negative edges as well. 
    In case of negative wt cycles, no solution exists for the SP problem; this algorithm will detect if
    such cycles exist. 
    """
    par = {v:None for v in g.graph.keys()}
    d = {v:sys.maxsize for v in g.graph.keys()}
    
    par[src], d[src] = src, 0
    nv = len(g.graph.keys())
    
    def relax(u,v):
        if d[v] > d[u] + g.weights[(u,v)]:
                d[v] = d[u] + g.weights[(u,v)]
                par[v] = u
    
    def is_neg_cycle(es):
        for e in es:
            u, v = e
            if d[v] > d[u] + g.weights[(u,v)]:
                return True
        return False
    
    def get_path(u, p):
        if u == src:
            p.append(u)
            return p[::-1]
        else:
            p.append(u)
            return get_path(par[u],p)
        
    es = g.weights.keys()
    for i in range(1,nv):
        for e in es:
            u, v = e
            relax(u,v)
        
    paths = {}
    negcycle = is_neg_cycle(es)
    
    if not negcycle:
        for v in g.graph.keys():
            if d[v] < sys.maxsize:
                path = get_path(v, [])
                paths[v] = (path,d[v])
    
    return negcycle, paths

In [28]:
invalid, paths = bellman_ford_sssp(g, 'a')

In [31]:
paths

{'a': (['a'], 0),
 'u': (['a', 'u'], 1),
 'v': (['a', 'v'], 1),
 'b': (['a', 'b'], 2),
 'w': (['a', 'v', 'w'], 3),
 'c': (['a', 'b', 'c'], 7),
 'x': (['a', 'v', 'w', 'x'], 7),
 'd': (['a', 'v', 'w', 'x', 'd'], 10)}

defaultdict(list,
            {'a': ['u', 'v', 'b'],
             'u': ['a', 'v'],
             'v': ['a', 'b', 'u', 'w'],
             'b': ['a', 'v', 'w', 'c'],
             'w': ['b', 'c', 'v', 'x'],
             'c': ['b', 'w', 'x', 'd'],
             'x': ['c', 'd', 'w'],
             'd': ['c', 'x']})